
# 📡 RF Kalibratie + Positievisualisatie (3× Raspberry Pi)

Deze notebook combineert:
1. **Kalibratie en ruisbepaling** van drie Raspberry Pi's  
2. **Interactieve visualisatie** van de Pi-posities en de geschatte GSM-positie met logaritmisch afstandsmodel  

De parameters (C, n, σ, ...) worden automatisch ingeladen uit `rf_calibration_params.json` en kunnen **niet aangepast** worden.  
Je kunt wél de posities van de Pi's en GSM met sliders wijzigen om de geometrie en cirkels te verkennen.


In [28]:

import json
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, fixed

# Inladen van eerder berekende parameters
with open("rssi_parameters.json", "r") as f:
    params = json.load(f)

combined = params["combined"]
per_pi = params["individual"]

print("✅ Parameters geladen uit rf_calibration_params.json")

# We tonen ze mooi in een tabelvorm
import pandas as pd
df_params = pd.DataFrame(per_pi).T
display(df_params.round(4))

print("\nGecombineerd model:")
for k, v in combined.items():
    print(f"{k:25s}: {v:.4f}")


✅ Parameters geladen uit rf_calibration_params.json


,path_loss_n,C_intercept,slope_c1,sigma_residual_db,R2
pi4,21.7189,-43.1889,-21.7189,3.6114,0.8106
pi5_rood,16.7963,-46.7490,-16.7963,3.2513,0.7595
pi5_zwart,22.8693,-40.7384,-22.8693,4.0045,0.7942



Gecombineerd model:
path_loss_n              : 20.4615
C_intercept              : -43.5588
slope_c1                 : -20.4615
sigma_residual_db        : 3.6224
R2                       : 0.7881


In [29]:

# Model: RSSI(d) = C - 10*n*log10(d)
def predicted_rssi(d, C, n):
    return C - 10 * n * np.log10(d)

# Omgekeerd: afstand uit RSSI
def distance_from_rssi(rssi, C, n):
    return 10 ** ((C - rssi) / (10 * n))


In [30]:
import numpy as np

def trilaterate(p1, p2, p3, r1, r2, r3):
    """
    Berekent (x, y) waar de drie cirkels met centra p1, p2, p3 en stralen r1, r2, r3 elkaar snijden.
    Geeft een benaderde oplossing via lineaire algebra (least squares).
    """
    # Omzetten naar numpy
    P1, P2, P3 = np.array(p1), np.array(p2), np.array(p3)

    # Formules gebaseerd op lineaire verschillen van cirkelvergelijkingen
    A = 2 * np.array([
        [P2[0] - P1[0], P2[1] - P1[1]],
        [P3[0] - P1[0], P3[1] - P1[1]],
    ])
    b = np.array([
        r1**2 - r2**2 - np.dot(P1, P1) + np.dot(P2, P2),
        r1**2 - r3**2 - np.dot(P1, P1) + np.dot(P3, P3),
    ])

    try:
        sol = np.linalg.lstsq(A, b, rcond=None)[0]
        return sol
    except np.linalg.LinAlgError:
        return np.mean([P1, P2, P3], axis=0)


In [ ]:
from ipywidgets import interact, FloatSlider

def plot_positions(pi1_x, pi1_y, pi2_x, pi2_y, pi3_x, pi3_y, gsm_x, gsm_y, n_factor):
    """
    Toon Pi's, GSM, afstandscirkels met foutmarge én rode 'estimated detection area'.
    Fout op de rode cirkelstraal wordt onder de grafiek weergegeven.
    n_factor = schaalfactor waarmee je n kunt aanpassen.
    """
    plt.figure(figsize=(6,6))
    pis = {
        "Pi1": (pi1_x, pi1_y),
        "Pi2": (pi2_x, pi2_y),
        "Pi3": (pi3_x, pi3_y),
    }
    gsm = (gsm_x, gsm_y)

    est_xs, est_ys, d_errs = [], [], []

    for name, (x, y) in pis.items():
        key_map = {"Pi1": "pi4", "Pi2": "pi5_rood", "Pi3": "pi5_zwart"}
        pkey = key_map[name]
        C_i = per_pi[pkey]["C_intercept"]
        n_i = per_pi[pkey]["path_loss_n"] * n_factor     # 🔧 schaalbare n
        sigma_i = per_pi[pkey]["sigma_residual_db"]

        # afstand tussen GSM en Pi
        d = np.sqrt((gsm_x - x)**2 + (gsm_y - y)**2)

        # ✅ logaritmisch foutmodel
        d_upper = d * 10 ** (sigma_i / (10 * n_i))
        d_lower = d * 10 ** (-sigma_i / (10 * n_i))
        d_err = (d_upper - d_lower) / 2.0                # halve spreiding
        d_errs.append(d_err)

        # tekenen
        label_text = f"{name} (n={n_i:.1f}, σ={sigma_i:.2f} dB)"
        plt.scatter(x, y, label=label_text, s=100)
        plt.gca().add_patch(plt.Circle((x, y), d, color="gray", alpha=0.25))
        plt.gca().add_patch(plt.Circle((x, y), d_upper, color="gray", alpha=0.1, linestyle="--"))
        plt.gca().add_patch(plt.Circle((x, y), d_lower, color="gray", alpha=0.1, linestyle="--"))
        plt.text(x + 0.1, y + 0.1, f"{name}", fontsize=8)
        est_xs.append(x)
        est_ys.append(y)

    # GSM
    plt.scatter(gsm_x, gsm_y, color="red", label="GSM", s=100, marker="x")
    plt.text(gsm_x + 0.1, gsm_y + 0.1, "GSM", color="red", fontsize=9)

    # 🔴 Trilateratie
    pi_coords = [pis["Pi1"], pis["Pi2"], pis["Pi3"]]
    radii = [np.sqrt((gsm_x - x)**2 + (gsm_y - y)**2) for (x, y) in pi_coords]
    est_cx, est_cy = trilaterate(pi_coords[0], pi_coords[1], pi_coords[2], *radii)
    est_r = np.mean(d_errs)

    plt.gca().add_patch(plt.Circle((est_cx, est_cy), est_r, color="red", alpha=0.15, lw=2))
    plt.text(est_cx + 0.2, est_cy + 0.2, "Estimated GSM zone", color="red", fontsize=8)

    plt.title("Positie van Pi's, GSM en geschatte detectiezone (met ruis)")
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")
    plt.axis("equal")
    plt.grid(True)
    plt.legend()
    plt.show()

    avg_err = np.mean(d_errs)
    print(f"Gemiddelde straalfout op geschatte detectiezone: ±{avg_err:.3f} meter (n-factor={n_factor:.2f})")
interact(
    plot_positions,
    pi1_x=FloatSlider(value=-1.0, min=-10, max=10, step=0.1, description="Pi1 x"),
    pi1_y=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi1 y"),
    pi2_x=FloatSlider(value=1.0, min=-10, max=10, step=0.1, description="Pi2 x"),
    pi2_y=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi2 y"),
    pi3_x=FloatSlider(value=0.0, min=-10, max=10, step=0.1, description="Pi3 x"),
    pi3_y=FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="Pi3 y"),
    gsm_x=FloatSlider(value=1.0, min=-10, max=10, step=0.1, description="GSM x"),
    gsm_y=FloatSlider(value=1.0, min=-10, max=10, step=0.1, description="GSM y"),
    n_factor=FloatSlider(value=1.0, min=0.1, max=2.0, step=0.05, description="n factor"), # 🔥 nieuw
);


interactive(children=(FloatSlider(value=-1.0, description='Pi1 x', max=10.0, min=-10.0), FloatSlider(value=0.0…


## 📘 Uitleg bij de visualisatie

- De **drie blauwe punten** zijn de Raspberry Pi’s.  
- De **rode ‘x’** is de GSM.  
- Elke **grijze cirkel** stelt de afstand voor tussen een Pi en de GSM.  
  (In een echte simulatie zouden die afstanden uit het RSSI-model komen.)

Met de **sliders** kun je de posities aanpassen en zien hoe de geometrie verandert.  
De parameters `C`, `n`, `σ` worden weergegeven maar niet gewijzigd — die komen uit de kalibratie.
